In [1]:
import pandas as pd
from nltk.corpus import stopwords
import re
import numpy
from keras.layers import Input, LSTM, Dense, Embedding, Dropout
import string
from gensim.models import KeyedVectors
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras import optimizers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk.tokenize import word_tokenize
from keras.models import Sequential

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def normalize_sentence(sentence):
	#sentence = string.decode('utf-8')
	sentence = (str(sentence)).lower()

	# unit
	#weight
	sentence = re.sub(r"(\d+)kgs ", lambda m: m.group(1) + ' kg ', sentence)	# e.g. 4kgs => 4 kg
	sentence = re.sub(r"(\d+)kg ", lambda m: m.group(1) + ' kg ', sentence)	 # e.g. 4kg => 4 kg
	sentence = re.sub(r"(\d+)k ", lambda m: m.group(1) + '000 ', sentence)	  # e.g. 4k => 4000
	#lenght
	sentence = re.sub(r"(\d+)km ", lambda m: m.group(1) + ' km ', sentence)
	sentence = re.sub(r"(\d+)cm ", lambda m: m.group(1) + ' cm ', sentence)
	sentence = re.sub(r"(\d+)m ", lambda m: m.group(1) + ' m ', sentence)
	#data
	sentence = re.sub(r"(\d+)gb ", lambda m: m.group(1) + ' gb ', sentence)
	#money
	sentence = re.sub(r"\$(\d+)", lambda m: m.group(1) + ' dollar ', sentence)
	sentence = re.sub(r"(\d+)\$", lambda m: m.group(1) + ' dollar ', sentence)

	# acronym
	sentence = re.sub(r"can\'t", "can not", sentence)
	sentence = re.sub(r"cannot", "can not ", sentence)
	sentence = re.sub(r"what\'s", "what is", sentence)
	sentence = re.sub(r"\'ve ", " have ", sentence)
	sentence = re.sub(r"n\'t", " not ", sentence)
	sentence = re.sub(r"i\'m", "i am ", sentence)
	sentence = re.sub(r"\'re", " are ", sentence)
	sentence = re.sub(r"\'d", " would ", sentence)
	sentence = re.sub(r"\'ll", " will ", sentence)
	sentence = re.sub(r" e mail ", " email ", sentence)
	sentence = re.sub(r" e \- mail ", " email ", sentence)
	sentence = re.sub(r"e-mail", " email ", sentence)
	sentence = re.sub(r",000", '000', sentence)
	sentence = re.sub(r"\'s", " ", sentence)
	
	# punctuation
	sentence = re.sub(r"\+", " + ", sentence)
	sentence = re.sub(r"'", " ", sentence)
	sentence = re.sub(r"\"", "", sentence)
	sentence = re.sub(r"-", " ", sentence)
	sentence = re.sub(r"/", " / ", sentence)
	sentence = re.sub(r"\\", " \ ", sentence)
	sentence = re.sub(r"=", " = ", sentence)
	sentence = re.sub(r"\^", " ^ ", sentence)
	sentence = re.sub(r":", " : ", sentence)
	sentence = re.sub(r"\.", "", sentence)
	sentence = re.sub(r",", " ", sentence)
	sentence = re.sub(r"\?", "", sentence)
	sentence = re.sub(r"!", "", sentence)
	sentence = re.sub(r"\"", " \" ", sentence)
	sentence = re.sub(r"&", " & ", sentence)
	sentence = re.sub(r"\|", " | ", sentence)
	sentence = re.sub(r";", " ; ", sentence)
	sentence = re.sub(r"\(", " ( ", sentence)
	sentence = re.sub(r"\)", " ) ", sentence)

	# symbol replacement
	sentence = re.sub(r"&", " and ", sentence)
	sentence = re.sub(r"\|", " or ", sentence)
	sentence = re.sub(r"\\", " or ", sentence)
	sentence = re.sub(r"/", " or ", sentence)
	sentence = re.sub(r"=", " equal ", sentence)
	sentence = re.sub(r"\+", " plus ", sentence)
	sentence = re.sub(r"₹", " rs ", sentence)
	sentence = re.sub(r"\$", " dollar ", sentence)
	sentence = re.sub(r"%", " percent ", sentence)

	while '  ' in sentence:
		sentence = re.sub(r"  ", " ", sentence)
	return sentence

In [3]:
english_stopwords = set(stopwords.words('english'))

In [4]:
total_dataset = pd.read_csv('train.csv').values[:,3:6]

In [5]:
total_dataset = numpy.asanyarray(sorted(total_dataset,key=lambda x: x[2]))

In [6]:
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300-SLIM.bin', binary=True)

In [7]:
train_positive = []
train_negative = []
test_positive = []
test_negative = []
i = 0
j = len(total_dataset) - 1
while (i < j):
    if total_dataset[i][2] == 0:
        if len(test_negative) < len(train_negative):
            test_negative.append(total_dataset[i])
        else:
            train_negative.append(total_dataset[i])
    if total_dataset[j][2] == 1:
        if len(test_positive) < len(train_positive):
            test_positive.append(total_dataset[j])
        else:
            train_positive.append(total_dataset[j])
    j = j - 1
    i = i + 1              

In [8]:
train_positive = train_positive[0:20000]
train_negative = train_negative[0:20000]
total_dataset = train_positive
total_dataset.extend(train_negative)
total_dataset = numpy.asanyarray(total_dataset)
test_positive = test_positive[0:20000]
test_negative = test_negative[0:20000]
total_dataset_test = test_positive
total_dataset_test.extend(test_negative)
total_dataset_test = numpy.asanyarray(total_dataset_test)

In [9]:
def normalize_sentence_list(list_sentence):
	for i in range (0,len(list_sentence)):
		list_sentence[i] = normalize_sentence(list_sentence[i])
	return list_sentence

def normalize_sentence_dataset(dataset):
	dataset[:,0] = normalize_sentence_list(dataset[:,0])
	dataset[:,1] = normalize_sentence_list(dataset[:,1])
	return dataset

def lemmatize_sentence_dataset(dataset):
	lemmatizer = WordNetLemmatizer()
	for idx_col in range(0,2):
		for idx_row in range (0,len(dataset[:,0])):  
			token_list = dataset[idx_row,idx_col].split(" ")
			dataset[idx_row,idx_col] = ""
			for i in range(0,len(token_list)):
				dataset[idx_row,idx_col] += lemmatizer.lemmatize(token_list[i])+" "
	return dataset

def remove_stop_word(sentence):
	for stopword in stopwords.words('english'):
			sentence = re.sub(r" "+stopword+" ", " ", sentence)
	return sentence
def get_avg_vector(sentence, model, num_features=300):
	sentence = sentence.strip().split(' ')
	sentence_avg_vec = numpy.zeros((num_features,),dtype="float32")
	n_vocab_word = 0
	for word in sentence:
		if (word in model.wv.vocab):
			n_vocab_word += 1
			sentence_avg_vec = numpy.add(sentence_avg_vec, model[word])
		elif (word.title() in model.wv.vocab):
			n_vocab_word += 1
			sentence_avg_vec = numpy.add(sentence_avg_vec, model[word.title()])
		elif (word.upper() in model.wv.vocab):
			n_vocab_word += 1
			sentence_avg_vec = numpy.add(sentence_avg_vec, model[word.upper()])
	if (n_vocab_word != 0):
		sentence_avg_vec = numpy.divide(sentence_avg_vec, n_vocab_word)
	return sentence_avg_vec

def get_number_count(sentence):
	count = 0
	for word in sentence.strip().split(' '):
		if (word.isdigit()):
			count += 1
	return count
	
def get_oov_count(sentence, model):
	count = 0
	sentence = sentence.strip().split(' ')
	for word in sentence:
		if ((word not in model.wv.vocab) and (word.title() in model.wv.vocab) and (word.upper() in model.wv.vocab)):
			count += 1
	return count
	
def get_features(data, model):
	q1 = get_avg_vector(data[0], model)
	q2 = get_avg_vector(data[1], model)
	n_w_q1 = len(data[0].strip().split(' '))
	n_w_q2 = len(data[1].strip().split(' '))
	list_features = []
	list_features.extend(q1)
	list_features.extend(q2)
	list_features.append(numpy.dot(q1, q2)/(numpy.linalg.norm(q1)* numpy.linalg.norm(q2)))
	list_features.append(n_w_q1)
	list_features.append(n_w_q2)
	list_features.extend([get_oov_count(data[1], model),get_oov_count(data[0], model),get_number_count(data[1]), abs(n_w_q1-n_w_q2)])
	list_features.extend(get_avg_vector(data[0].split(' ')[0], model))
	list_features.extend(get_avg_vector(data[1].split(' ')[0], model))
	return list_features

In [10]:
total_dataset = normalize_sentence_dataset(total_dataset)
total_dataset_test = normalize_sentence_dataset(total_dataset_test)

In [11]:
total_dataset = lemmatize_sentence_dataset(total_dataset)
total_dataset_test = lemmatize_sentence_dataset(total_dataset_test)

In [12]:
for data in total_dataset:
    data[0] = remove_stop_word(data[0])
    data[1] = remove_stop_word(data[1])
    
for data in total_dataset_test:
    data[0] = remove_stop_word(data[0])
    data[1] = remove_stop_word(data[1])

In [13]:
temp_data = []
for data in total_dataset:
    if data[0] is not None and data[1] is not None:
        if all(c in string.printable for c in data[0]) and all(c in string.printable for c in data[1]):
            temp_data.append(data)
total_dataset = numpy.asanyarray(temp_data)
temp_data = []
for data in total_dataset_test:
    if data[0] is not None and data[1] is not None:
        if all(c in string.printable for c in data[0]) and all(c in string.printable for c in data[1]):
            temp_data.append(data)
total_dataset_test = numpy.asanyarray(temp_data)

In [14]:
temp_data = []
for data in total_dataset:
    temp_data.append([get_features(data, word2vec), data[2]])
total_dataset = numpy.asanyarray(temp_data)
temp_data = []
for data in total_dataset_test:
    temp_data.append([get_features(data, word2vec), data[2]])
total_dataset_test = numpy.asanyarray(temp_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in float_scalars


In [15]:
train_x = [x[0] for x in total_dataset]
train_y = [x[1] for x in total_dataset]
test_x = [x[0] for x in total_dataset_test]
test_y = [x[1] for x in total_dataset_test]

In [ ]:
# model = Sequential()
# model.add(Embedding(len(train_x[0]), output_dim=100))
# model.add(LSTM(50))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
# model.fit(train_x, train_y, epochs=50000, validation_split=0.2)

In [16]:
import math
train_xx = []
train_yy = []
for i,a in enumerate(train_x):
    if all(not math.isnan(c) and not math.isinf(c) for c in train_x[i]):
        train_xx.append(train_x[i])
        train_yy.append(train_y[i])
test_xx = []
test_yy = []
for i,a in enumerate(test_x):
    if all(not math.isnan(c) and not math.isinf(c) for c in test_x[i]):
        test_xx.append(test_x[i])
        test_yy.append(test_y[i])

In [24]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(train_xx)
train_xx = scaler.transform(train_xx)
test_xx = scaler.transform(test_xx)

In [ ]:
model = Sequential()
model.add(Dense(input_dim=len(train_xx[0]), units=150, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adagrad(lr=0.001), metrics=['accuracy'])
model.fit(train_xx, train_yy, epochs=200, validation_split=0.2)

Train on 31351 samples, validate on 7838 samples
Epoch 1/200
31351/31351 [==============================] - 11s 342us/step - loss: 0.6640 - acc: 0.6174 - val_loss: 0.9370 - val_acc: 0.1180
Epoch 2/200
31351/31351 [==============================] - 10s 306us/step - loss: 0.6441 - acc: 0.6478 - val_loss: 0.9073 - val_acc: 0.1951
Epoch 3/200
31351/31351 [==============================] - 9s 298us/step - loss: 0.6341 - acc: 0.6598 - val_loss: 0.9254 - val_acc: 0.2057
Epoch 4/200
31351/31351 [==============================] - 9s 276us/step - loss: 0.6305 - acc: 0.6633 - val_loss: 0.9277 - val_acc: 0.2177
Epoch 5/200
31351/31351 [==============================] - 10s 317us/step - loss: 0.6261 - acc: 0.6687 - val_loss: 0.9147 - val_acc: 0.2383
Epoch 6/200
31351/31351 [==============================] - 9s 290us/step - loss: 0.6240 - acc: 0.6711 - val_loss: 0.9094 - val_acc: 0.2450
Epoch 7/200
31351/31351 [==============================] - 9s 291us/step - loss: 0.6223 - acc: 0.6702 - val_loss: 

In [50]:
pred_y = model.predict_classes(test_xx)

19628/19628 [==============================] - 2s 106us/step


In [ ]:
from sklearn import svm

In [ ]:
cls = svm.SVC(verbose=10)
cls.fit(numpy.asanyarray(train_xx), numpy.asanyarray(train_yy))

In [ ]:
pred_y = cls.predict(test_xx)

In [51]:
from sklearn.metrics import f1_score
f1_score(pred_y, test_yy, average='weighted')

0.72507827307976369

ImportError: `save_weights` requires h5py.